In [1]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
import cv2
from PIL import Image as im
from PIL.ImageShow import IPythonViewer
from scipy import ndimage
from matplotlib.pyplot import figure
from glob import glob
from tqdm import tqdm

from typing import *

In [2]:
def binarize(pic):
    wd, ht = pic.size
    pixels = np.array(pic.convert('L').getdata(), np.uint8)
    bin_img = (1 - pixels.reshape((ht, wd)) / 255.0)
    #plt.imshow(bin_img, cmap='gray')
    return bin_img

In [3]:
def build_grid(image):
    bin_img = binarize(image)
    height, width = bin_img.shape
    
    horizontal = [0]
    encountered = False
    for idx in range(height):
        row = bin_img[idx]
        if np.sum(row) > 1:
            encountered = True
        elif encountered == True and np.sum(bin_img[idx+20, :]) < 1:
            if idx - horizontal[-1] < 20: 
                horizontal[-1] = idx
            else:
                horizontal.append(idx)
            encountered = False

    vertical = [0]
    bin_img_t = bin_img.T
    encountered = False
    for idx in range(width):
        col = bin_img_t[idx]
        if np.sum(col) > 1:
            encountered = True
        elif encountered == True and np.sum(bin_img_t[idx+20, :]) < 1:
            if idx - vertical[-1] < 20: 
                vertical[-1] = idx
            else:
                vertical.append(idx)
            encountered = False
    
    if not len(vertical) == 7 or not len(horizontal) == 12:
        print(f"{font}: vertical - {vertical}, horizontal - {horizontal}")
    assert len(vertical) == 7 # 6 vertical lines after each column + left
    assert len(horizontal) == 12 # 11 horizontal lines under each row + top
    
    return vertical, horizontal

In [5]:
def put_in_a_box(char):
    bin_img = binarize(char)
    height, width = bin_img.shape
    def find_border(bin_img):
        for (idx, row) in enumerate(bin_img):
            if np.sum(row) > 1:
                return idx
    
    left = find_border(bin_img.T)
    top = find_border(bin_img)
    right = width - find_border(bin_img.T[::-1, :])
    bottom = height - find_border(bin_img[::-1, :])
    
    return char.crop((left, top, right, bottom))
    

In [6]:
def expand2square(pil_img, background_color):
    width, height = pil_img.size
    if width == height:
        return pil_img
    elif width > height:
        result = im.new(pil_img.mode, (width, width), background_color)
        result.paste(pil_img, (0, (width - height) // 2))
        return result
    else:
        result = im.new(pil_img.mode, (height, height), background_color)
        result.paste(pil_img, ((height - width) // 2, 0))
        return result

In [250]:
def add_margin(pil_img, top, right, bottom, left, color):
    width, height = pil_img.size
    new_width = width + right + left
    new_height = height + top + bottom
    result = im.new(pil_img.mode, (new_width, new_height), color)
    result.paste(pil_img, (left, top))
    return result

In [270]:
def pretty(char):
    box = put_in_a_box(char)
    #square = expand2square(box, (255,255,255))
    #w, h = square.size
    #top_and_bot = (120-h)//2
    #l_and_r = (120-w)//2
    #return add_margin(square, top_and_bot, l_and_r, top_and_bot, l_and_r, (255,255,255))
    return 

In [7]:
fonts = list(filter(lambda n: n.endswith("png"), os.listdir()))
for font in tqdm(fonts):
    img = im.open(font)
    bin_img = binarize(img)
    vertical, horizontal = build_grid(img)

    letters = (x for x in "АБВГДЕЁЖЗИЙабвгдеёжзийКЛМНОПРСТУФклмнопрстуфХЦЧШЩЪЫЬЭЮЯхцчшщъыьэюя")
    os.makedirs(f"t/{font[:-4]}", mode=0o0777, exist_ok=True)

    for left, right in [(vertical[i], vertical[i+1]) for i in range(len(vertical)-1)]:
        height, _ = bin_img.shape
        cropped = img.crop((left, 0, right, height))
        for upper, lower in [(horizontal[i], horizontal[i+1]) for i in range(len(horizontal)-1)]:
            width, _  = cropped.size
            char = cropped.crop((0, upper, width, lower))
            char.save(f"t/{font[:-4]}/{next(letters)}.png")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:02<00:00,  5.88it/s]
